## Module 4 Final Project Submission - Computer Vision

* Student name: Emily J. Cain
* Student pace: Full Time
* Scheduled project review date/time: 
* Instructor name: Rafael Carrasco
* Blog post URL: https://medium.com/@magicmilly


## I. Load & Organize Data
## II. Inspection & Exploratory Data Analysis
## III. Preprocessing
## IV. Modeling 
## V. Model Evaluation
## VI. Conclusions & Recommendations

In [5]:
import datetime
import numpy as np
import os
import pandas as pd
import scipy
# import shutil
import time
import warnings
# import zipfile

from keras import layers
from keras import models 
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image
from scipy import ndimage

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
warnings.filterwarnings(action='ignore', category=FutureWarning)

## I. Load & Organize Data

In [11]:
# calculate amount of training images
cnv_train_imgs = len(os.listdir('Data/train/CNV'))
dme_train_imgs = len(os.listdir('Data/train/DME'))
drusen_train_imgs = len(os.listdir('Data/train/DRUSEN'))
normal_train_imgs = len(os.listdir('Data/train/NORMAL'))

total_train_imgs = cnv_train_imgs + dme_train_imgs + drusen_train_imgs + normal_train_imgs
print(f'There are {total_train_imgs} training images.')

There are 83484 training images.
